In [ ]:
%%script echo skipping
# above line causes this cell to not run

%pip install pandas numpy scikit-learn

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd

from config import Config
from setup import features, windows, zeek

In [ ]:
# run_id = "220126_100532"

pcap_path = Path(Config.PCAP_PATH).resolve()
run_id = zeek.process_pcap(pcap_path)

flows, dns_events_by_host = features.parse_run(run_id)
rows = windows.create_windows(flows)

df = pd.DataFrame(rows)
df.head()


### Data Cleaning

### Feature Selection

### Model Parameters

In [ ]:
# research which other models to try
from sklearn.ensemble import IsolationForest

In [ ]:
model = IsolationForest(
    n_estimators=300,
    contamination=contamination,
    random_state=42,
    n_jobs=-1,
)

pipe = Pipeline(steps=[("prep", preprocess), ("model", model)])


### Training Pipeline

In [ ]:
pipe.fit(X)

pred = pipe.predict(X)  # -1 anomaly, +1 normal
score = pipe.decision_function(X)  # higher=more normal, lower=more anomalous

labels = np.where(pred == -1, "malicious", "benign")

pred_df = df.copy()
pred_df["label"] = labels
pred_df["anomaly_score"] = score

pred_df[["label"]].value_counts()